# BPZ RAIL - DP0.1

no bringing to memory yet

## Imports

### common libs

In [ ]:
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline 

### RAIL

RAIL is an LSST-DESC software created to process different algorithms used to calculate photometric redshift. Its main goal is to minimize impact that different infrastructures can cause on different algorithms, for that it unifyes in a modular code supporting different inputs that different algorithms needs and padronizing the output so that it can be a more fair comparison between their results.

Rail uses 4 principal libraries in its core: <br>
_tables_io_: for data manipulation as hdf5 files, fits, etc. <br>
_qp_: used to parametrize data PDFs for metrics calculation. <br>
_ceci_: construct pipelines, produces a .yaml within the steps and configurations as threads. <br>
_pzflow_: creates a flow for data creation. <br>

#### Core.
Where the main functions are going to manage the data and files that the program creates. It works based in the behavioral [chain of resposability](https://refactoring.guru/pt-br/design-patterns/chain-of-responsibility) pattern , where you create a flux in the code, where there is a request related/processed by a class handler that decides to pass it foward or not according to what is defined. So for that, what bpz does is create a class request (eg: Inform_BPZ_lite) that has all the inputs/configurations and is handled by its class handler (BPZ_lite).

#### Creation.
Contains all the support for data creation, as degradors, data flow creation, Column remapping, etc. It creates .hdf5 files with the data that is being manipulated.

#### Estimation.
This is where the photo-z codes are defined and executed.  <br>
_inform_: this is where the PRIORS for template fitting are informed and the machine learning codes are trained. <br>
_estimate_: where the algorith is executed though the .evaluate() function.
The code is wrapped as a RAIL stage so that it can be run in a controlled way. Estimation code can be stored in a yaml file to be run as a ceci module.


#### Evaluation.
This step contais the metrics for performance of the estimated codes.
<br>
------
For installation instructions check the official documentation: https://lsstdescrail.readthedocs.io/en/latest/source/installation.html <br>
For Rail versions check: https://github.com/LSSTDESC/RAIL/releases

Before running RAIL, make sure it is up to date: 
```pip install rail --upgrade ``` 

In [ ]:
import rail
import qp
import tables_io

from rail.core.data import TableHandle, DataStore
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

##from rail.creation.engines.flowEngine import FlowEngine, FlowPosterior

from rail.estimation.algos.bpz_lite import Inform_BPZ_lite, BPZ_lite

from rail.evaluation.evaluator import Evaluator

#for rail versions
help(rail)

## General Configs

Setting some default number of rows for pandas. So that it doesnt display all of them. 

In [ ]:
pd.set_option('display.max_rows', 20)

Defining some variables that will help us with directories. 

In [ ]:
CURR_DIR = os.getcwd()
RAIL_DIR = os.path.join(os.path.dirname(rail.__file__), '..')
CURR_DIR, RAIL_DIR

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

## Reading DP0.1 csv

In [ ]:
interesting_headers = [
"coadd_objects_id",
"ra",
"dec",
"mag_g",
"magerr_g",
"mag_i",
"magerr_i",
"mag_r",
"magerr_r",
"mag_u",
"magerr_u",
"mag_y",
"magerr_y",
"mag_z",
"magerr_z",
"z_true"
]

trainFile = os.path.join(CURR_DIR, 'dp0_train_random.csv')
full_data = pd.read_csv(trainFile, usecols=interesting_headers)

In [ ]:
type(full_data)

In [ ]:
size = len(full_data)//2

train = full_data.sample(n=size,random_state=1, ignore_index=True)
test = full_data.drop(train.index)

In [ ]:
train

In [ ]:
test

---

##  RAIL BPZ

### Core - Data Storage 

Basically Rail store data in a transient class DataStore. This class associate keys and products in a dictionary, so that when program needs to excecute some step it has the functions that read, writes, and a data handlers.

A DataHandler basically is a class that act like a handler for some data. What it does is that it associates the data with a file and the tool to read the file. The DataStore stores those handlers and their files associated with a key. So that when the algorithms process they are can propperly read the file content.

In [ ]:
columns_remmap = {
"coadd_objects_id": "id",
"ra": "coord_ra",
"dec": "coord_dec",
"mag_g": "mag_g_lsst",
"magerr_g": "mag_err_g_lsst",
"mag_i": "mag_r_lsst",
"magerr_i": "mag_err_r_lsst",
"mag_r": "mag_i_lsst",
"magerr_r": "mag_err_i_lsst",
"mag_u": "mag_u_lsst",
"magerr_u": "mag_err_u_lsst",
"mag_y": "mag_y_lsst",
"magerr_y": "mag_err_y_lsst",
"mag_z": "mag_z_lsst",
"magerr_z": "mag_err_z_lsst",
"z_true": "redshift"
}


col_remapper_train = ColumnMapper.make_stage(name='col_remapper_train', columns=columns_remmap)
table_conv_train = TableConverter.make_stage(name='table_conv_train', output_format='numpyDict')

results_remmaped = col_remapper_train(train)
train_data = table_conv_train(results_remmaped)

In [ ]:
type(train_data.data)

As we can see, ceci stages basically configures the name and some configuration, so that when the stage runs, it return a TableHander, such as a PqHandler, Hdf5Handle or FitsHandle. 

obs: For machine leaning algorithms if may be necessary to configure a flowHandler too.

In [ ]:
type(results_remmaped.data)

In [ ]:
type(train_data.data)

In [ ]:
full_sample = tables_io.convertObj(train_data.data, tables_io.types.PD_DATAFRAME)
type(full_sample)

Here we should have somewhere a redshift result from other surveys.

### PRIORS - Inform

In [ ]:
columns_file = os.path.join(CURR_DIR, 'configs/bpz.columns')
inform_bpz = Inform_BPZ_lite.make_stage(
    name='inform_bpzlite', 
    input="output_table_conv_train", #"inprogress_output_table_conv_train.hdf5", 
    model='trained_BPZ_output.pkl', ##não precisaria isso pro bpz
    hdf5_groupname='', 
    columns_file=columns_file
)
inform_bpz.config.to_dict()

In [ ]:
%%time
returned = inform_bpz.inform(train_data)

___

## Posterior -> Estimate


In [ ]:
results_remmaped = col_remapper_train(test)
test_data = table_conv_train(results_remmaped)

In [ ]:
estimate_bpz = BPZ_lite.make_stage(
    name='estimate_bpz', 
    hdf5_groupname='', 
    columns_file=columns_file, 
    model=inform_bpz.get_handle('model'))
estimate_bpz.is_parallel()

In [ ]:
estimate_bpz.config_options['data_path'].value

In [ ]:
bpz_estimated = estimate_bpz.estimate(test_data)

In [ ]:
bpz_estimated().build_tables()

results_tables = tables_io.convertObj(bpz_estimated().build_tables()['ancil'], tables_io.types.PD_DATAFRAME)
results_tables

In [ ]:
test_data_orig = test_data.data

evaluator = Evaluator.make_stage(name=f'bpz_eval', truth=test_data_orig)
result_dict = evaluator.evaluate(bpz_estimated, test_data_orig)

In [ ]:
results_tables = tables_io.convertObj(result_dict.data, tables_io.types.PD_DATAFRAME)
results_tables.head()

___
## VOU MEXER AINDA - Resultado pz x spec-z

In [ ]:
zmode = bpz_estimated().ancil['zmode']

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['redshift'],zmode,s=1,c='k',label='simple bpz mode')
plt.plot([0,3],[0,3],'r--');
plt.xlabel("true redshift")
plt.ylabel("bpz photo-z")